In [1]:
import requests
import os
from bs4 import BeautifulSoup
from datetime import date
import datetime
from faker import Faker
import random
import boto3
import time
import json
import datetime
import uuid
fake = Faker()


In [2]:
resp = requests.get('https://www.4icu.org/top-universities-north-america/')
soup = BeautifulSoup(resp.text, 'html.parser')
rows = soup.findAll('tr')
school_list = []
for tr in rows:
    cols = tr.findAll('td')
    for col in cols:
        if col:
            link = col.find('a')
            if link:
                school_list.append(link.contents[0])

In [3]:
path = '/Users/syeehyn/Desktop/spring2020/coms6998/final/YouTutor-frontEnd/dev'
college_major = json.load(open(os.path.join(path, 'collage_major.json')))
grad_major = json.load(open(os.path.join(path,'grad_major.json')))

In [4]:
school_list = school_list[:50]
genders = ['F', 'M']
degrees = ['B.S', 'B.A', 'PhD', 'M.E', 'M.F.A', 'M.S', 'M.A']
languages = ['spanish', 'english', 'japanese', 'chinese', 'french']
hours = range(8, 20, 1)
IMG_API = 'https://fakeface.rest/face/json?maximum_age=29&minimum_age=18&'
female_query = 'gender=female'
male_query = 'gender=male'

In [12]:
def random_user():
    user = {}
    tags = []
    user['user_id'] = fake.email()
    user['gender'] = random.choice(genders)
    if user['gender'] == 'F':
        user['first_name']= fake.first_name_female()
        if random.choice([1, 0]):
            user['picture'] = requests.get(IMG_API+female_query).json()['image_url']
        else:
            user['picture'] = None
    else:
        user['first_name']= fake.first_name_male()
        if random.choice([1, 0]):
            user['picture'] = requests.get(IMG_API+male_query).json()['image_url']
        else:
            user['picture'] = None
    user['last_name'] = fake.language_name()
    user['tutor'] = random.choice([True, False])
    user['school'] = random.choice(school_list)
    user['degree'] = random.choice(degrees)
    if user['degree'] in ['B.S', 'B.A']:
        user['major'] = random.choice(college_major)
    else:
        user['major'] = random.choice(grad_major)
    if random.choice([0,0,0,0,1]):
        tags.append(user['major'])
    if random.choice([0,0,0,0,1]):
        tags.append(random.choice(languages))
        if random.choice([0,0,0,0,1]):
            tags.append(random.choice(languages))

    intro = f'''Hello, I am {user['first_name']}, a {user['degree']} student major in {user['major']} from {user['school']}
    '''.replace('\n', '')

    user['introduction'] = intro
    user['tags'] = tags

    if user['tutor']:
        availability = []
        for _ in range(random.randint(0, 5)):
            time_stamp = sorted(random.choices(hours, k=2))
            while len(set(time_stamp)) == 1:
                time_stamp = sorted(random.choices(hours, k=2))
            time_stamp = '-'.join([str(i)+':00' for i in time_stamp])
            time_stamp = time_stamp + ',' + fake.day_of_week()
            availability.append(time_stamp)
        user['availability'] = availability
    else:
        user['availability'] = []
    return json.loads(json.dumps(user))

In [13]:
TABLE_NAME = 'user-profile'

def create_user_table():
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    try:
        if dynamodb.Table(TABLE_NAME).table_status == 'ACTIVE':
            return dynamodb.Table(TABLE_NAME)
    except:
        pass
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[
            {
                'AttributeName': 'user_id',
                'KeyType': 'HASH'  # Partition key
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'user_id',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        }
    )
    time.sleep(5)
    return table

In [14]:
def generate_users(const):
    users = []
    for _ in range(const):
        users.append(random_user())
    return users
def write_users(table, users):
    with table.batch_writer() as batch:
        for user in users:
            batch.put_item(
                Item={
                    'user_id': user['user_id'],
                    'gender': user['gender'],
                    'first_name' : user['first_name'],
                    'last_name': user['last_name'],
                    'tutor': user['tutor'],
                    'school': user['school'],
                    'degree': user['degree'],
                    'major': user['major'],
                    'introduction': user['introduction'],
                    'tags': user['tags'],
                    'availability': user['availability'],
                    'picture': user['picture']
                }
            )

In [15]:
CONST = 100
table = create_user_table()
users = generate_users(CONST)
write_users(table, users)

In [16]:
with open('users.json', 'w') as f:
    json.dump(users, f)

In [17]:
myself = {
    'user_id': 'sy2938@columbia.edu',
    'gender': 'M',
    'first_name' : 'Jackie',
    'last_name': 'Yuan',
    'tutor': True,
    'school': 'Columbia University',
    'degree': 'M.S',
    'major': 'Data Science',
    'introduction': 'Hello, I am Jackie, a M.S student major in Data Science from Columbia University',
    'tags': ['Data Science','Chinese'],
    'availability': ['9:00-13:00,Thursday',
                  '10:00-17:00,Monday',
                  '12:00-13:00,Saturday',
                  '9:00-13:00,Friday',
                  '12:00-13:00,Sunday'],
    'picture': 'https://content.fakeface.rest/male_20_cb646615bf253a1351b1a04c44248768d565cbff.jpg'
}

In [18]:
table.put_item(
    Item={
        'user_id': 'sy2938@columbia.edu',
        'gender': 'M',
        'first_name' : 'Jackie',
        'last_name': 'Yuan',
        'tutor': True,
        'school': 'Columbia University',
        'degree': 'M.S',
        'major': 'Data Science',
        'introduction': 'Hello, I am Jackie, a M.S student major in Data Science from Columbia University',
        'tags': ['Data Science','Chinese'],
        'availability': ['9:00-13:00,Thursday',
                      '10:00-17:00,Monday',
                      '12:00-13:00,Saturday',
                      '9:00-13:00,Friday',
                      '12:00-13:00,Sunday'],
        'picture': 'https://content.fakeface.rest/male_20_cb646615bf253a1351b1a04c44248768d565cbff.jpg'
    }
)

{'ResponseMetadata': {'RequestId': '7AV9VCSK4K9L7FR4U4RHOCC9KRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 09 Apr 2021 06:40:20 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '7AV9VCSK4K9L7FR4U4RHOCC9KRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [19]:
users = json.load(open('users.json'))

In [20]:
time.mktime(datetime.datetime(2021, 4, 
random.choice(range(1,30)), 
random.choice(range(0, 24)) , random.choice(range(0, 61))).timetuple())

1619158440.0

In [25]:
def random_question(user, tutor=None):
    question = {}
    question['question_id'] = str(uuid.uuid1())
    question['user_id'] = user['user_id']
    
    title = f'What is {fake.bs()}'
    question['title'] = title
    question['detail'] = f'I have question regarding to {title}, can anyone help?'
    if user['tags'] != []:
        tags = random.choices(user['tags'], k=random.choice(range(len(user['tags']))))
    else:
        tags = []
    question['tags'] = tags
    if random.choice([1,0]):
        question['attachment'] = fake.url()
    if tutor:
        question['tutor_id'] = tutor['user_id']
        question['question_status'] = random.choice(['accepted', 'confirmed'])
        date, hour, length = random.choice(range(5, 30)), random.choice(hours), random.choice(range(1, 4))
        question['start_time'] = datetime.datetime(2021, 4, 
                        date, hour , 0).isoformat()
        question['end_time'] = datetime.datetime(2021, 4, 
                        date, hour+length , 0).isoformat()
        question['created_at'] = int(time.mktime(datetime.datetime(2021, 4, 
                                                    date - random.choice(range(1,5)), 
                                                   random.choice(range(0, 24)) , 0).timetuple()))
        
    else:
        question['tutor_id'] = 'Null'
        question['question_status'] = random.choice(['posted', 'posted', 'posted', 'posted', 'canceled'])
        question['start_time'] = 'Null'
        question['end_time'] = 'Null'
        question['created_at'] = int(time.mktime(datetime.datetime(2021, 4, 
                                random.choice(range(1,30)), 
                               random.choice(range(0, 24)) , random.choice(range(0, 60))).timetuple()))
    
    
    return json.loads(json.dumps(question))

In [36]:
Q_TABLE_NAME = 'question'
def create_question_table():
    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
    try:
        if dynamodb.Table(Q_TABLE_NAME).table_status == 'ACTIVE':
            return dynamodb.Table(Q_TABLE_NAME)
    except:
        pass
    table = dynamodb.create_table(
        TableName=Q_TABLE_NAME,
        KeySchema=[
            {
                'AttributeName': 'question_id',
                'KeyType': 'HASH'  # Partition key
            }, 
            {
                'AttributeName': 'created_at',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'question_id',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'user_id',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'tutor_id',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'created_at',
                'AttributeType': 'N'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 10,
            'WriteCapacityUnits': 10
        },
        GlobalSecondaryIndexes = [
            {
                'IndexName': 'user_id',
                'KeySchema': [
                    {
                        'AttributeName': 'user_id',
                        'KeyType': 'HASH'
                    },
                ],
                'Projection': {
                'ProjectionType': 'ALL'
                },
                'ProvisionedThroughput': {
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
                }
            },
            {
                'IndexName': 'tutor_id',
                'KeySchema': [
                    {
                        'AttributeName': 'tutor_id',
                        'KeyType': 'HASH'
                    },
                ],
                'Projection': {
                'ProjectionType': 'ALL',
                },
                'ProvisionedThroughput': {
                'ReadCapacityUnits': 10,
                'WriteCapacityUnits': 10
                }
            }
        ]
    )
    time.sleep(5)
    return table

In [37]:
table = create_question_table()

In [38]:
CONST = 25
questions = []
for _ in range(CONST):
    if random.choice([1, 0]):
        user, tutor = random.choices(users, k=2)
        questions.append(random_question(user, tutor))
    else:
        user = random.choice(users)
        questions.append(random_question(user, None))
for _ in range(CONST):
    if random.choice([1, 0]):
        if random.choice([1, 0]):
            tutor = random.choice(users)
            questions.append(random_question(myself, tutor))
        else:
            questions.append(random_question(myself, None))
    else:
        user = random.choice(users)
        questions.append(random_question(user, myself))

In [39]:
def write_questions(table, questions):
    s_q = sorted(questions, key = lambda x: x['created_at'])
    with table.batch_writer() as batch:
        for question in s_q:
            batch.put_item(
                question
            )

In [42]:
write_questions(table, questions)